<a href="https://colab.research.google.com/github/rahulvenkat207/PDF_Retriver/blob/main/pdfRetrievel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face Integration for Document Retrieval and Question Answering in LangChain

# importing modules

In [26]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

# uploading files


In [3]:
from google.colab import files
uploaded = files.upload()


Saving acsbr-015.pdf to acsbr-015 (5).pdf


In [4]:
# Load PDF documents from the uploaded folder
loader = PyPDFDirectoryLoader(".")
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(documents)

# Print the first document chunk (if it exists)
if final_documents:
    print("First document chunk:")
    print(final_documents[0])
else:
    print("No document chunks found.")

First document chunk:
page_content='Poverty in States and Metropolitan  \nAreas: 2022\nAmerican Community Survey Briefs\nDecember 2023ACSBR-016By Craig Benson\nINTRODUCTION\nPlanners, policymakers, and community stakeholders \nuse poverty estimates as key indicators to evaluate trends and current economic conditions within com-munities and to make comparisons across demo-graphic groups. Federal and state governments often \nuse these estimates to allocate funds to local com-\nmunities. Government agencies, researchers, and local organizations regularly use these estimates to identify the number of individuals and families eligible for vari-ous programs and to measure economic well-being.\nThis brief uses the 2021 and 2022 American \nCommunity Survey (ACS) 1-year estimates and the \n2021 and 2022 Puerto Rico Community Surveys' metadata={'source': 'acsbr-016.pdf', 'page': 0}


## Embedding Using Huggingface

In [5]:

huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when p

In [6]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-1.00630280e-02 -3.98545153e-02 -2.14797407e-02  5.86761385e-02
  8.02909061e-02  5.45422360e-02 -1.89205445e-02 -3.84308654e-03
 -5.06586060e-02 -9.22424626e-03  5.20436279e-02  2.59992275e-02
 -1.59694124e-02 -1.32009666e-02 -3.79712917e-02 -2.48674899e-02
 -4.57806960e-02  5.27276583e-02  2.56090928e-02  1.23642618e-02
  5.47023602e-02 -9.55430232e-03 -7.26953298e-02 -1.91431381e-02
  5.93195856e-02 -3.60356346e-02 -1.51885775e-04 -5.13967834e-02
 -4.05045003e-02 -1.59561872e-01  6.18918706e-03  2.72258744e-03
  4.80102375e-02 -6.29636412e-03 -3.37924738e-03 -4.26316075e-02
  3.80311124e-02  5.58205210e-02  8.53926782e-03  5.33879139e-02
 -4.03712387e-04 -2.85844766e-02 -1.75250154e-02 -3.33270915e-02
 -6.52636513e-02 -6.25040084e-02 -1.69884469e-02 -3.36706918e-03
 -5.80814444e-02 -1.87587962e-02  3.09326444e-02 -1.79333314e-02
  1.11763366e-02  4.28264476e-02  4.17709574e-02  2.02438459e-02
  9.16245393e-03 -3.49307843e-02 -2.00626235e-02  4.42196578e-02
  3.16111781e-02  9.09475

# Document Vectorization using FAISS with Hugging Face Embeddings

In [7]:
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [8]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


## Environment secret keys

In [9]:
from google.colab import userdata
sec_key=userdata.get("HUGGINGfACEKEY")


In [10]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"]=sec_key

In [11]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Define the HuggingFace model for LLM

In [28]:
hf = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"temperature": 0.7, "max_new_tokens": 100}
)

# PromptTemplate

In [29]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [14]:
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])


# Create a retriever

In [19]:
retriever = vectorstore.as_retriever()

# Set up the RetrievalQA

In [20]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [24]:
query="""WHAT IS HEALTH INSURANCE COVERAGE"""

In [25]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased direct